In [ ]:
import pandas as pd

In [ ]:
ks200 = pd.read_csv('./dataset/kospi200.csv', index_col=0)

In [ ]:
# 음수 값이 있을수 있으므로 절댓값 취하기
ks200 = ks200.abs()

In [ ]:
# 날짜 7월 31일부터 -> 옵션은 전날 종가로 기준 행사가를 잡음
ks200 = ks200.loc[ks200['일자'] > 20230727]

In [ ]:
# 필요없는 컬럼 제거
ks200.drop(columns=['대업종구분', '소업종구분', '종목정보', '전일종가'], inplace=True)

In [ ]:
# 소수점 생성
ks200[['현재가', '시가', '고가', '저가']] = ks200[['현재가', '시가', '고가', '저가']] / 100

In [ ]:
ks200.head(3)

##### 여기까지 정제된 데이터프레임으로 거래대금 거래량 확인

In [ ]:
df = ks200[['일자', '현재가']]

In [ ]:
df

In [ ]:
# 소수점만 가져오기
result = df['현재가'].apply(lambda x: x % 10)

In [ ]:
result = pd.DataFrame(result)
result.columns = ['소수']

In [ ]:
price = pd.concat([df, result], axis=1)

In [ ]:
price.head(3)

In [ ]:
# 행사가 컬럼 생성
price['행사가'] = 0

In [ ]:
# 소수에 따라 기준 행사가 지정
for index, i in enumerate(price['소수']):
	if i > 7.5:
		price['행사가'][index] = 7.5
	elif i > 5:
		price['행사가'][index] = 5
	elif i > 2.5:
		price['행사가'][index] = 2.5
	else:
		price['행사가'][index] = 0

In [ ]:
price['기준행사가'] = price['행사가'] + (price['현재가'].apply(lambda x: x // 10) * 10)

In [ ]:
price

In [ ]:
strike_price = price[['일자', '기준행사가']]

In [ ]:
strike_price['S-4'] = strike_price['기준행사가'] - 10
strike_price['S-3'] = strike_price['기준행사가'] - 7.5
strike_price['S-2'] = strike_price['기준행사가'] - 5
strike_price['S-1'] = strike_price['기준행사가'] - 2.5
strike_price['S+1'] = strike_price['기준행사가'] + 2.5
strike_price['S+2'] = strike_price['기준행사가'] + 5
strike_price['S+3'] = strike_price['기준행사가'] + 7.5
strike_price['S+4'] = strike_price['기준행사가'] + 10

In [ ]:
strike_price.info()

In [ ]:
strike_price = strike_price.set_index('일자')

In [76]:
# 전날 종가가 당일 기준행사가이므로 행 밀어주기
strike_price = strike_price.shift(-1)

In [84]:
strike_price.drop(index=[20230728, 20231023, 20231024], axis=0, inplace=True)

In [86]:
strike_price.to_csv('./dataset/strike_price.csv')

##### 해당 날짜별 기준행사가 및 (+-4)

In [87]:
strike_price

,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
일자,,,,,,,,,
20231020,322.5,312.5,315.0,317.5,320.0,325.0,327.5,330.0,332.5
20231019,327.5,317.5,320.0,322.5,325.0,330.0,332.5,335.0,337.5
20231018,325.0,315.0,317.5,320.0,322.5,327.5,330.0,332.5,335.0
20231017,322.5,312.5,315.0,317.5,320.0,325.0,327.5,330.0,332.5
20231016,325.0,315.0,317.5,320.0,322.5,327.5,330.0,332.5,335.0
20231013,327.5,317.5,320.0,322.5,325.0,330.0,332.5,335.0,337.5
20231012,322.5,312.5,315.0,317.5,320.0,325.0,327.5,330.0,332.5
20231011,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5
20231010,317.5,307.5,310.0,312.5,315.0,320.0,322.5,325.0,327.5


In [90]:
oct = strike_price[:9]
sep = strike_price[9:25]
aug = strike_price[25:44]
july = strike_price[44:]

In [92]:
# 해당 달 별 최소 행사가와 최대 행사가 찾기
oct.describe()

,기준행사가,S-4,S-3,S-2,S-1,S+1,S+2,S+3,S+4
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,323.055556,313.055556,315.555556,318.055556,320.555556,325.555556,328.055556,330.555556,333.055556
std,3.703414,3.703414,3.703414,3.703414,3.703414,3.703414,3.703414,3.703414,3.703414
min,317.500000,307.500000,310.000000,312.500000,315.000000,320.000000,322.500000,325.000000,327.500000
25%,322.500000,312.500000,315.000000,317.500000,320.000000,325.000000,327.500000,330.000000,332.500000
50%,322.500000,312.500000,315.000000,317.500000,320.000000,325.000000,327.500000,330.000000,332.500000
75%,325.000000,315.000000,317.500000,320.000000,322.500000,327.500000,330.000000,332.500000,335.000000
max,327.500000,317.500000,320.000000,322.500000,325.000000,330.000000,332.500000,335.000000,337.500000
